In [42]:


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import zipfile
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping

<h5>Here I used kaggle dataset.
Please change the paths accordingly.

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dogs-vs-cats/sampleSubmission.csv
/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip


In [34]:
class LoadData():
    def __init__(self):
        self.train_file = None
        self.test_file = None
        self.submisson_File = None
        self.submisson_File = os.path.join('/kaggle/input/dogs-vs-cats',"Submission.csv")
        self.data_frame = None
        self.train_df = None
        self.validation_df = None
        
        zip_ref = zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip', 'r')
        zip_ref.extractall('/tmp/train')
        self.train_dir = '/tmp/train/train'
        zip_ref.close()
        
    def prepare_data(self):
        filenames = os.listdir("/tmp/train/train")
        categories = []
        for filename in filenames:
            category = filename.split('.')[0]
            if category == 'dog':
                categories.append(1)
            else:
                categories.append(0)

        self.data_frame = pd.DataFrame({'filename': filenames,'category': categories})
        self.data_frame["category"] = self.data_frame["category"].replace({0: 'cat', 1: 'dog'}) 
        self.train_df, self.validation_df = train_test_split(self.data_frame, test_size=0.20, random_state=42)
        self.train_df = self.train_df.reset_index(drop=True)
        self.validation_df = self.validation_df.reset_index(drop=True)


In [35]:
ld_obj = LoadData()
ld_obj.prepare_data()
ld_obj.data_frame.head()

,filename,category
0,dog.890.jpg,dog
1,dog.1178.jpg,dog
2,dog.7845.jpg,dog
3,dog.4632.jpg,dog
4,cat.3660.jpg,cat


In [36]:
ld_obj.train_df.head()

,filename,category
0,dog.10381.jpg,dog
1,dog.1864.jpg,dog
2,cat.3131.jpg,cat
3,dog.12317.jpg,dog
4,dog.2524.jpg,dog


In [39]:
ld_obj.validation_df.head()

,filename,category
0,dog.7293.jpg,dog
1,dog.1822.jpg,dog
2,dog.1683.jpg,dog
3,cat.3994.jpg,cat
4,dog.12248.jpg,dog


In [40]:
class PreProcessing():
    def __init__(self):
        self.train_data_gen = None
        self.valid_data_gen = None
        self.FAST_RUN = False
        self.image_width=128
        self.image_height=128
        self.image_size=(self.image_width, self.image_height)
        self.image_channels=3
        self.batch_size = 15
        self.train_data_length = ld_obj.train_df.shape[0]
        self.validation_data_length = ld_obj.validation_df.shape[0]
        self.train_generator = None
        self.validation_generator = None
        
    def data_genrator(self):
        train_datagen = ImageDataGenerator(
        rotation_range=15,
        rescale=1./255,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1
        )

        self.train_generator = train_datagen.flow_from_dataframe(
            ld_obj.train_df, 
            "/tmp/train/train/", 
            x_col='filename',
            y_col='category',
            target_size=self.image_size,
            class_mode='categorical',
            batch_size=self.batch_size
        )
        
        validation_datagen = ImageDataGenerator(rescale=1./255)
        self.validation_generator = validation_datagen.flow_from_dataframe(
            ld_obj.validation_df, 
            "/tmp/train/train/", 
            x_col='filename',
            y_col='category',
            target_size=self.image_size,
            class_mode='categorical',
            batch_size=self.batch_size
        )


In [41]:
pre_process_obj = PreProcessing()
pre_process_obj.data_genrator()

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


In [46]:
class DesignModel():
    def __init__(self):
        self.model = None
        
    def create_model(self):
        model = Sequential()

        model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(pre_process_obj.image_width, pre_process_obj.image_height, pre_process_obj.image_channels)))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        
        model.add(Dense(2, activation='softmax'))
        self.model = model
    
    def compile_model(self):
        self.model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

        self.model.summary()
    def train_model(self):
        
        self.model.fit(pre_process_obj.train_generator,
                              validation_data=pre_process_obj.validation_generator,
                              steps_per_epoch=pre_process_obj.train_data_length//pre_process_obj.batch_size,
                              epochs=15,
                              validation_steps=pre_process_obj.validation_data_length//pre_process_obj.batch_size,
                              verbose=2)
    

In [ ]:
model_obj = DesignModel()
model_obj.create_model()
model_obj.compile_model()
model_obj.train_model()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_4 (Batch (None, 126, 126, 32)      128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_5 (Batch (None, 61, 61, 64)        256       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 30, 64)       

In [ ]:
class Prediction():
    def __init__(self):
        self.model = model
    def predict(self):
        